In [55]:
import json
import os
import pandas as pd
import pyshark
import random
import re
from collections import OrderedDict
from json import JSONDecoder
from json import loads
from json import dumps
from os.path import isfile, join
from os import listdir
from pprint import pprint
from subprocess import Popen, PIPE, check_output

# Extract funtion is used to flatten nested dictionaries to a Lvl 1 Dictionary
# Cons: It does not avoid collisions
def extract(dict_in, dict_out):
    for key, value in dict_in.iteritems():
        if isinstance(value, dict): # If value itself is dictionary
            extract(value, dict_out)
        elif isinstance(value, unicode):
            # Write to dict_out
            dict_out[key] = value
    return dict_out

# Parse Object Pairs function identifies all keys that are equal at the same level
# and calls make_unique to rename the keyname
def parse_object_pairs(pairs):
    dct = OrderedDict()
    #dct = dict()
    for key, value in pairs:
        if key in dct:
            key = make_unique(key, dct)
        dct[key] = value
    return dct

# Make Unique function is in charge of renaming keys which have the same name 
def make_unique(key, dct):
    counter = 0
    unique_key = key

    while unique_key in dct:
        counter += 1
        unique_key = '{}-{}'.format(key, str(counter).zfill(2))
    return unique_key


# Parse Object Pairs 2 function identifies all keys that are equal at the same level
# and calls make_unique to rename the keyname
# In addition, it will tag the -00 suffix to each label making data more homogeneous
def parse_object_pairs2(pairs):
    dct = OrderedDict()
    dummy_dct = OrderedDict()
    #print("PAIRS:")
    #print(pairs)
    #print("\n")
    for key, value in pairs:
        #print("Make Unique Loop")
        key = make_unique2(key, dct, dummy_dct)
        dct[key] = value
    return dct

# Make Unique 2 function is in charge of renaming keys which have the same name 
def make_unique2(key, dct, dummy_dct):
    counter = 0
    unique_key = key
    #print("Counter %s" %counter)
    #print("Unique Key:")
    #print(unique_key)
    #print("Dictionary")
    #print(dct)
    if unique_key not in dummy_dct:
        #print("*** IF Loop ***")
        #print(dct)
        #print("Pre-Unique: %s" %unique_key)
        unique_key = '{}-{}'.format(key, str(counter).zfill(2))
        #print("Pos-Unique: %s" %unique_key)
        #print("*** Dummy DCT ***")
        dummy_dct[key] = ""
        #print(dummy_dct)
    else:
        while unique_key in dct or unique_key in dummy_dct:
            #print("*** Else Loop ***")
            #print(dct)
            #print("Pre-Unique: %s" %unique_key)
            unique_key = '{}-{}'.format(key, str(counter).zfill(2))
            #print("Pos-Unique: %s" %unique_key)
            counter += 1
    #print("\n")
    return unique_key

# Flatten DIct function is in charge of flattening the nested dictionaries
# While flattening, it will rename the keys at the last level by prepending predecesor key names
def flatten_dict(d, prefix='>'):
    def items():
        # A clojure for recursively extracting dict like values
        for key, value in d.items():
            if isinstance(value, dict):
                for sub_key, sub_value in flatten_dict(value).items():
                    # Key name should imply nested origin of the dict,
                    # so we use a default prefix of __ instead of _ or .
                    if key == "_source-00" or key == "layers-00":
                        yield sub_key, sub_value
                    else:
                        yield key + prefix + sub_key, sub_value
            else:
                yield key, value
    return dict(items())

# Define the Main Directory containing all uncompressed PCAPS
pcaps_dir='/home/cc/Contagio'
# Create a list of the Sub-directories within the Main Directory
directories = [x[0] for x in os.walk(pcaps_dir)]

# Print list of Sub-directories
print(directories)
uncpx_pcaps_dir = directories

# Define required empty lists
failed_pcaps=[]

# Define variables
mw_id_counter = 0

# Create List of Unique IDs to identify to assign to each Malware PCAP
#mw_random_ids = sorted(random.sample(range(10000,20000), 1300))
mw_random_ids = list(range(10000, 14000))
#mw_id = mw_random_ids[0]

# Dictionary for matching Malware ID and Malware Name
mw_id_dict = {}

#Create Empty Dataframe
columns = ['mw_id']
df_ = pd.DataFrame(index=[0], columns=columns)

# List of frames summary valies
summary_values = ['number', 'time', 'source', 'destination', 'protocol', 'length', 'info', 'mw_id']

# The following loop will perform the following actions:
# 1) Loop through every Sub-directory
# 2) Loop through every PCAP within each Sub-directory
# 3) Read every PCAP using tshark filtering only frames containing ssl traffic
#    In addtion, the output of all protocol features will be delivered in a nested JSON format
# 4) The extracted dictionary will be parsed using parse_object_pairs and flatten_dict function
for pcap_dir in uncpx_pcaps_dir[0:1]: #uncpx_pcaps_dir[0:1]
    pcap_dir = "/home/cc/Contagio/Exploit_Kits_and_Malware"
    print("\n")
    print(pcap_dir)
    pcap_dir_files = [f for f in listdir(pcap_dir) if isfile(join(pcap_dir, f))]
    print("\n")
    pcap_dir_files = ["EK_MALWARE_2014-04-29-fake-flash-updater-02_mailware-traffic-analysis.net.pcap"]
    #pcap_dir_files = ["EK_MALWARE_2014-02-23-Neutrino-EK-traffic_mailware-traffic-analysis.net.pcap"]
    pkt_num = 0
    for i_cap in pcap_dir_files:
        # Get a new malware ID
        mw_id = mw_random_ids[mw_id_counter]
        # Save Malware Id and Name to Dictionary
        mw_id_dict[mw_id] = i_cap
        print("ID: %s" %mw_id)
        print("MW: %s" %mw_id_dict[mw_id])
        # Increase Counter
        mw_id_counter = mw_id_counter + 1
        
        # Print PCAP
        cap_temp = pyshark.FileCapture(pcap_dir+'/'+i_cap, keep_packets=False, only_summaries=True)
        #print(cap_temp)
        
        for i_pkt in cap_temp:
            #print(i_pkt)
            #i_pkt_layers = []
            #i_pkt_layers = re.findall(r'<(\w+)', str(i_pkt.layers))
            #pkt_layers_full = []
            #for i_layer in i_pkt_layers:
            #    pkt_layers_full.append(i_layer)
            #pkt_num = pkt_num+1
            #print(pkt_layers_full)
            #print(dir(i_pkt))
            #print(i_pkt.no)
            #print(i_pkt.time)
            #print(i_pkt.source)
            #print(i_pkt.destination)
            #print(i_pkt.protocol)
            #print(i_pkt.length)
            #print(i_pkt.info)
            temp_ipkt_list = [i_pkt.no, i_pkt.time, i_pkt.source, i_pkt.destination, i_pkt.protocol, i_pkt.length, i_pkt.info, mw_id]
            # Create Temp Dataframe
            df = pd.DataFrame([temp_ipkt_list], index=[1], columns=summary_values)
            # Append Temp Dataframe to Main Dataframe (df_)
            df_ = df_.append([df], ignore_index=True)
'''
            try:
                print("--- ETH LAYER ---")
                #pprint(i_pkt.eth._all_fields)
                #for key in i_pkt.eth._all_fields.keys():
                #    eth_list.append(key)
            except:
                pass
            try:
                print("--- IP LAYER ---")
                #pprint(i_pkt.ip._all_fields)
                print("IP Source")
                pprint(i_pkt.ip.src)
                print("IP Destination")
                pprint(i_pkt.ip.dst)
                print("IP Protocol")
                pprint(i_pkt.ip.proto)
                #for key in i_pkt.ip._all_fields.keys():
                #    ip_list.append(key)
            except:
                pass
            try:
                print("--- UDP LAYER ---")
                #pprint(i_pkt.udp._all_fields)
                print("UDP Source Port")
                pprint(i_pkt.udp.srcport)
                print("UDP Destination Port")
                pprint(i_pkt.udp.dstport)               
                #for key in i_pkt.udp._all_fields.keys():
                #    udp_list.append(key)
            except:
                pass
            try:
                print("--- TCP LAYER ---")
                #pprint(i_pkt.tcp._all_fields)
                print("TCP Source Port")
                pprint(i_pkt.tcp.srcport)
                print("TCP Destination Port")
                pprint(i_pkt.tcp.dstport) 
                #for key in i_pkt.tcp._all_fields.keys():
                #    tcp_list.append(key)
            except:
                pass
'''

Exception ignored in: <bound method Capture.__del__ of <FileCapture /home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2014-02-23-Neutrino-EK-traffic_mailware-traffic-analysis.net.pcap (0 packets)>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/pyshark/capture/capture.py", line 422, in __del__
    self.close()
  File "/usr/local/lib/python3.5/dist-packages/pyshark/capture/capture.py", line 412, in close
    self.eventloop.run_until_complete(self._close_async())
  File "/usr/local/lib/python3.5/dist-packages/trollius/base_events.py", line 300, in run_until_complete
    return future.result()
  File "/usr/local/lib/python3.5/dist-packages/trollius/futures.py", line 287, in result
    raise self._exception
  File "/usr/local/lib/python3.5/dist-packages/trollius/tasks.py", line 251, in _step
    result = coro.throw(exc)
  File "/usr/local/lib/python3.5/dist-packages/pyshark/capture/capture.py", line 417, in _close_async
    yield From(self._cleanup_su

['/home/cc/Contagio', '/home/cc/Contagio/CRIME', '/home/cc/Contagio/CRIME/tbotpcaps', '/home/cc/Contagio/GENERAL_DEFCON_17', '/home/cc/Contagio/Exploits_and_Scans', '/home/cc/Contagio/General', '/home/cc/Contagio/APT', '/home/cc/Contagio/Exploit_Kits_and_Malware', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2015-01-08-pcap-files-associated-with-malware-from-82.244.160.22_mailware-traffic-analysis.net', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2014-08-13-sandbox-analysis-pcap-files_mailware-traffic-analysis.net']


/home/cc/Contagio/Exploit_Kits_and_Malware


ID: 10000
MW: EK_MALWARE_2014-04-29-fake-flash-updater-02_mailware-traffic-analysis.net.pcap


'\n            try:\n                print("--- ETH LAYER ---")\n                #pprint(i_pkt.eth._all_fields)\n                #for key in i_pkt.eth._all_fields.keys():\n                #    eth_list.append(key)\n            except:\n                pass\n            try:\n                print("--- IP LAYER ---")\n                #pprint(i_pkt.ip._all_fields)\n                print("IP Source")\n                pprint(i_pkt.ip.src)\n                print("IP Destination")\n                pprint(i_pkt.ip.dst)\n                print("IP Protocol")\n                pprint(i_pkt.ip.proto)\n                #for key in i_pkt.ip._all_fields.keys():\n                #    ip_list.append(key)\n            except:\n                pass\n            try:\n                print("--- UDP LAYER ---")\n                #pprint(i_pkt.udp._all_fields)\n                print("UDP Source Port")\n                pprint(i_pkt.udp.srcport)\n                print("UDP Destination Port")\n                pp

In [63]:
# Create list of discovered features
total_features = list(df_.columns.values)
print ("\n Number of Features:")
print(len(total_features))
#pprint(total_features)
#df_.drop(df_.index[[0]], inplace=True)
df_ = df_.reindex(columns=['mw_id','number','time','source','destination','protocol','length','info'])
pd.set_option('display.max_colwidth', -1,'display.max_rows', None)
df_


 Number of Features:
8


,mw_id,number,time,source,destination,protocol,length,info
1,10000,2,0.123809,172.16.223.2,172.16.223.132,DNS,97,Standard query response 0x3b13 A webradio-powerplay.de A 88.198.55.148
2,10000,3,0.224012,172.16.223.132,88.198.55.148,TCP,66,49175 \xe2\x86\x92 80 [SYN] Seq=0 Win=8192 Len=0 MSS=1460 WS=4 SACK_PERM=1
3,10000,4,0.370391,88.198.55.148,172.16.223.132,TCP,60,"80 \xe2\x86\x92 49175 [SYN, ACK] Seq=0 Ack=1 Win=64240 Len=0 MSS=1460"
4,10000,5,0.370439,172.16.223.132,88.198.55.148,TCP,54,49175 \xe2\x86\x92 80 [ACK] Seq=1 Ack=1 Win=64240 Len=0
5,10000,6,0.370976,172.16.223.132,88.198.55.148,HTTP,736,GET / HTTP/1.1
6,10000,7,0.371101,88.198.55.148,172.16.223.132,TCP,60,80 \xe2\x86\x92 49175 [ACK] Seq=1 Ack=683 Win=64240 Len=0
7,10000,8,0.55788,88.198.55.148,172.16.223.132,HTTP,559,HTTP/1.1 302 Found (text/html) (text/html)
8,10000,9,0.560402,172.16.223.132,88.198.55.148,HTTP,811,GET /news.php HTTP/1.1
9,10000,10,0.560527,88.198.55.148,172.16.223.132,TCP,60,80 \xe2\x86\x92 49175 [ACK] Seq=506 Ack=1440 Win=64240 Len=0
10,10000,11,1.215749,88.198.55.148,172.16.223.132,TCP,1422,[TCP segment of a reassembled PDU]


In [64]:
# Create mw_id_series
mw_id_series = pd.Series(mw_id_dict)
# Save mw_id_series
mw_id_series.to_pickle('mw_id_series.p')
# Save SSL df (df_)
df_.to_pickle('mw_dataframe.p')
# Save SSL features list
ssl_features_file = open('ssl_features.txt', 'w')
for item in total_features:
    ssl_features_file.write("%s\n" % item)

In [65]:
# Pretty function walks a dictionary and prints it based with specific number of spaces per level
def pretty(d, indent=0):
    for key, value in d.items():
        print('    ' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('    ' * (indent+1) + str(value))

# Pretty function walks a dictionary and prints it based with specific number of spaces per level
def pretty_save(d, out_tree, indent=0):
        for key, value in d.items():
            out_tree.write('    ' * indent + str(key)+"\n")
            if isinstance(value, dict):
                pretty_save(value, out_tree, indent+1)
            else:
                out_tree.write('    ' * (indent+1) + str(value)+"\n")
            
# SortOD function read a dictionary and sorts keys in alphabetical order per level          
def sortOD(od):
    res = OrderedDict()
    for k, v in sorted(od.items()):
        if isinstance(v, dict):
            res[k] = sortOD(v)
        else:
            res[k] = v
    return res

# Variable for creating an Ordered Dictionary for Features
features_tree = OrderedDict()

# Read the list of features and transfer them to an ordered dictionary
for feature_item in total_features:
    t = features_tree
    for part in feature_item.split('>'):
        t = t.setdefault(part, {})

# Sort features in alphabetical order
features_tree_ordered = sortOD(features_tree)

# Print Ordered Dictionary in Pretty Mode
#pretty(features_tree_ordered)

# Save SSL features tree list
with open('ssl_features_tree.txt', 'w') as out_tree:
    pretty_save(features_tree_ordered, out_tree)
out_tree.close()